In [1]:
%env CUDA_VISIBLE_DEVICES=''

import sys
sys.path.append('..')
sys.path.append('/scratch/mmd/chemprop/')

%load_ext autoreload

env: CUDA_VISIBLE_DEVICES=''


In [9]:
%autoreload

import pickle, torch

from molecule_ge_coordinated_embeddings.models import load_chemprop_model
from chemprop.utils import save_checkpoint

### From Original Chemprop Checkpoint

In [46]:
def load_strip_store(inpath, outpath):
    _, C, args = load_chemprop_model(
        inpath, load_weights=False, return_raw=True, use_cuda=False
    )
    save_checkpoint(outpath, C, None, None, args)
    return C

In [47]:
C = load_strip_store(
    '/crimea/molecule_ge_embedder/chemprop_models/model_unoptimized.pt',
    '/crimea/molecule_ge_embedder/runs/transfer_to_chemprop_runs/chemprop_unoptimized_random_init.pt'
)

### From Molecule Embedder

In [19]:
def load_from_molecule_embedder_checkpoint(checkpoint_path, orig_chemprop_path, outpath):
    ordered_state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    prefix = 'chemprop_encoder.'
    chemprop_encoder_state_dict = {
        k.replace(prefix, ''): v for k, v in ordered_state_dict.items() if k.startswith(prefix)
    }
    
    _, orig_model, args = load_chemprop_model(
        orig_chemprop_path, load_weights=False, return_raw=True, use_cuda=False
    )
    
    orig_model.encoder.load_state_dict(chemprop_encoder_state_dict)
    
    save_checkpoint(outpath, orig_model, None, None, args)

In [23]:
load_from_molecule_embedder_checkpoint(
    '/crimea/molecule_ge_embedder/runs/transfer_to_chemprop_runs/exp_1/molecule_embedder_exp/'
    '35c2bd2edeb5f11fa53f9241478ee399/checkpoint_35c2bd2edeb5f11fa53f9241478ee399_val_mrr=0.011574793016316887.pth',
    '/crimea/molecule_ge_embedder/chemprop_models/model_unoptimized.pt',
    '/crimea/molecule_ge_embedder/runs/transfer_to_chemprop_runs/chemprop_unoptimized_molembed_init_12_ep.pt'
)